In [1]:
import tkinter as tk
import sqlite3

def create_people_data():
    # Словарь с данными о людях
    people_data = {
        1: {'имя': 'Иван', 'фамилия': 'Петров', 'возраст': 30},
        2: {'имя': 'Сергей', 'фамилия': 'Сидоров', 'возраст': 25},
        3: {'имя': 'Владимир', 'фамилия': 'Иванов', 'возраст': 40}
    }

    try:
        # Создание подключения к базе данных
        conn = sqlite3.connect('people.db')

        # Создание курсора для выполнения SQL-запросов
        cursor = conn.cursor()

        # Создание таблицы, если она не существует
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS people (
                id INTEGER PRIMARY KEY,
                имя TEXT,
                фамилия TEXT,
                возраст INTEGER
            )
        ''')

        # Вставка данных в таблицу
        for person_id, person_data in people_data.items():
            cursor.execute('''
                INSERT INTO people (id, имя, фамилия, возраст)
                VALUES (?, ?, ?, ?)
            ''', (person_id, person_data['имя'], person_data['фамилия'], person_data['возраст']))

        # Применение изменений
        conn.commit()

        # Закрытие подключения к базе данных
        conn.close()

    except sqlite3.Error as e:
        print(f"An error occurred: {e}")

def add_person():
    # Получение данных из полей ввода
    имя = entry_имя.get()
    фамилия = entry_фамилия.get()
    возраст = entry_возраст.get()

    try:
        # Проверка наличия данных
        if not имя or not фамилия or not возраст:
            raise ValueError("All fields must be filled")

        # Создание подключения к базе данных
        conn = sqlite3.connect('people.db')

        # Создание курсора для выполнения SQL-запросов
        cursor = conn.cursor()

        # Вставка новой записи в таблицу
        cursor.execute('''
            INSERT INTO people (имя, фамилия, возраст)
            VALUES (?, ?, ?)
        ''', (имя, фамилия, возраст))

        # Применение изменений
        conn.commit()

        # Очистка полей ввода
        entry_имя.delete(0, tk.END)
        entry_фамилия.delete(0, tk.END)
        entry_возраст.delete(0, tk.END)

        # Обновление списка людей
        update_people_list()

        # Закрытие подключения к базе данных
        conn.close()

    except (sqlite3.Error, ValueError) as e:
        error_label.config(text=str(e))

def update_people_list():
    # Очистка списка людей
    listbox.delete(0, tk.END)

    try:
        # Создание подключения к базе данных
        conn = sqlite3.connect('people.db')

        # Создание курсора для выполнения SQL-запросов
        cursor = conn.cursor()

        # Получение данных из таблицы
        cursor.execute('SELECT * FROM people')
        people = cursor.fetchall()

        # Вывод названий столбцов
        listbox.insert(tk.END, f"{'ID':<5}{'Name':<20}{'Age':<5}")
        listbox.insert(tk.END, '-'*30)

        # Вывод данных в виджет списка
        for person in people:
            person_id = f"{person[0]:<5}"
            full_name = f"{person[1]} {person[2]:<20}"
            age = f"{person[3]:<5}"
            listbox.insert(tk.END, f"{person_id}{full_name}{age}")

        # Закрытие подключения к базе данных
        conn.close()

    except sqlite3.Error as e:
        error_label.config(text=str(e))

def exit_app():
    window.destroy()

# Создание окна
window = tk.Tk()

# Создание фрейма для центрирования
frame = tk.Frame(window)
frame.pack(expand=True)

# Создание виджета списка с увеличенной областью
listbox = tk.Listbox(frame, width=40, height=10)
listbox.pack()

# Выравнивание текста по левому краю
listbox.config(justify=tk.LEFT)

# Создание метки и полей ввода для добавления новых людей
label_имя = tk.Label(frame, text="Имя:")
label_имя.pack()

entry_имя = tk.Entry(frame)
entry_имя.pack()

label_фамилия = tk.Label(frame, text="Фамилия:")
label_фамилия.pack()

entry_фамилия = tk.Entry(frame)
entry_фамилия.pack()

label_возраст = tk.Label(frame, text="Возраст:")
label_возраст.pack()

entry_возраст = tk.Entry(frame)
entry_возраст.pack()

button_add = tk.Button(frame, text="Добавить человека", command=add_person)
button_add.pack()

error_label = tk.Label(frame, fg="red")
error_label.pack()

button_exit = tk.Button(frame, text="Выход", command=exit_app)
button_exit.pack()

try:
    # Создание подключения к базе данных
    conn = sqlite3.connect('people.db')

    # Создание курсора для выполнения SQL-запросов
    cursor = conn.cursor()

    # Создание таблицы, если она не существует
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS people (
            id INTEGER PRIMARY KEY,
            имя TEXT,
            фамилия TEXT,
            возраст INTEGER
        )
    ''')

    # Применение изменений
    conn.commit()

    # Создание записей о людях
    create_people_data()

    # Обновление списка людей
    update_people_list()

    # Закрытие подключения к базе данных
    conn.close()

except sqlite3.Error as e:
    print(f"An error occurred: {e}")

# Настройка выравнивания фрейма
frame.pack_configure(anchor=tk.CENTER)

# Запуск главного цикла окна
window.mainloop()
